Task 3: Implement Byte-Pair Encoding (BPE) and WordPiece
tokenizers from scratch, using the provided documents as your
training corpus.

In [11]:
import re
from collections import Counter, defaultdict
import os

In [12]:
from collections import Counter, defaultdict

class BPETokenizer:
    def __init__(self, vocab_size=30000):
        self.vocab_size = vocab_size
        self.vocab = []
        self.merges = {}

    def get_word_freqs(self, corpus):
        full_text = " ".join(corpus)
        words = re.findall(r'\w+|[^\w\s]', full_text.lower())
        return Counter(words)

    def train(self, corpus):
        word_freqs = self.get_word_freqs(corpus)
        alphabet = set()

        for word in word_freqs:
            alphabet.update(list(word))
        self.vocab = sorted(alphabet)

        splits = {word: list(word) + ['</w>'] for word in word_freqs}

        while len(self.vocab) < self.vocab_size:
            pair_freqs = defaultdict(int)
            for word, freq in word_freqs.items():
                word_splits = splits[word]
                for i in range(len(word_splits) - 1):
                    pair = (word_splits[i], word_splits[i+1])
                    pair_freqs[pair] += freq

            if not pair_freqs:
                break

            best_pair = max(pair_freqs, key=pair_freqs.get)
            merged_token = "".join(best_pair)
            self.vocab.append(merged_token)
            self.merges[best_pair] = merged_token

            for word in word_freqs:
                word_splits = splits[word]
                new_split = []
                i = 0
                while i < len(word_splits):
                    if i < len(word_splits) - 1 and (word_splits[i], word_splits[i+1]) == best_pair:
                        new_split.append(merged_token)
                        i += 2
                    else:
                        new_split.append(word_splits[i])
                        i += 1
                splits[word] = new_split

    def tokenize(self, text):
        text = text.lower()
        words = re.findall(r'\w+|[^\w\s]', text)

        tokenized_output = []
        for word in words:
            if word in self.vocab:
                tokenized_output.append(word)
                continue

            word_splits = list(word) + ['</w>']
            for pair, merge in self.merges.items():
                new_split = []
                i = 0
                while i < len(word_splits):
                    if i < len(word_splits) - 1 and (word_splits[i], word_splits[i+1]) == pair:
                        new_split.append(merge)
                        i += 2
                    else:
                        new_split.append(word_splits[i])
                        i += 1
                word_splits = new_split

            tokenized_output.extend(word_splits)

        return tokenized_output


In [13]:
class WordPieceTokenizer:
    def __init__(self, vocab_size=30000):
        self.vocab_size = vocab_size
        self.vocab = []
        self.unknown_token = "[UNK]"

    def get_word_freqs(self, corpus):
        full_text = " ".join(corpus)
        words = re.findall(r'\w+|[^\w\s]', full_text.lower())
        return Counter(words)

    def train(self, corpus):
        word_freqs = self.get_word_freqs(corpus)

        alphabet = set()
        for word in word_freqs:
            alphabet.update(list(word))
        self.vocab = sorted(alphabet) + [self.unknown_token]

        splits = {word: list(word) for word in word_freqs}

        while len(self.vocab) < self.vocab_size:
            scores = defaultdict(float)
            pair_freqs = defaultdict(int)
            char_freqs = defaultdict(int)

            for word, freq in word_freqs.items():
                word_splits = splits[word]
                for i in range(len(word_splits) - 1):
                    pair = (word_splits[i], word_splits[i+1])
                    pair_freqs[pair] += freq
                    char_freqs[word_splits[i]] += freq
                if word_splits:
                    char_freqs[word_splits[-1]] += freq

            for pair, freq in pair_freqs.items():
                if char_freqs[pair[0]] > 0 and char_freqs[pair[1]] > 0:
                    scores[pair] = freq / (char_freqs[pair[0]] * char_freqs[pair[1]])

            if not scores:
                break

            best_pair = max(scores, key=scores.get)
            first, second = best_pair
            merged_token = first + second.replace('##', '')
            merged_token_with_prefix = '##' + merged_token if not first.startswith('##') else merged_token

            self.vocab.append(merged_token)
            self.vocab.append(merged_token_with_prefix)

            for word in word_freqs:
                word_splits = splits[word]
                new_split = []
                i = 0
                while i < len(word_splits):
                    if i < len(word_splits) - 1 and (word_splits[i], word_splits[i+1]) == best_pair:
                        current_merged = word_splits[i] + word_splits[i+1].replace('##', '')
                        new_split.append(current_merged)
                        i += 2
                    else:
                        new_split.append(word_splits[i])
                        i += 1
                splits[word] = new_split

        self.vocab = sorted(set(self.vocab))

    def tokenize(self, text):
        text = text.lower()
        words = re.findall(r'\w+|[^\w\s]', text)

        tokenized_output = []
        for word in words:
            if word in self.vocab:
                tokenized_output.append(word)
                continue

            subwords = []
            current_pos = 0
            while current_pos < len(word):
                found_subword = None
                prefix = '##' if current_pos > 0 else ''

                for end_pos in range(len(word), current_pos, -1):
                    sub = prefix + word[current_pos:end_pos]
                    if sub in self.vocab:
                        found_subword = sub
                        break

                if found_subword:
                    subwords.append(found_subword)
                    current_pos += len(found_subword) - len(prefix)
                else:
                    subwords.append(self.unknown_token)
                    current_pos += 1

            tokenized_output.extend(subwords)

        return tokenized_output


In [14]:
K = 500

def load_documents_from_directory(directory_path):
    documents = []
    try:
        file_list = sorted(os.listdir(directory_path))
    except (IOError, AttributeError, FileNotFoundError):
        return []
    for filename in file_list:
        if filename.endswith(".txt"):
            filepath = os.path.join(directory_path, filename)
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                documents.append(f.read())
    return documents

bbc_folder_path = 'BBC'
corpus = load_documents_from_directory(bbc_folder_path)

if not corpus:
    print(f"Could not find or read files in directory '{bbc_folder_path}'.")
else:
    test_sentences = [
        "Learning models need lots of data.",
        "Tokenization helps language models handle new words."
    ]

    print("="*60)
    print("Byte-Pair Encoding (BPE) Tokenizer")
    print("="*60)
    bpe_tokenizer = BPETokenizer(vocab_size=K)
    bpe_tokenizer.train(corpus)

    print("\nBPE Tokenization Examples:")
    for sentence in test_sentences:
        tokens = bpe_tokenizer.tokenize(sentence)
        print(f"Sentence: '{sentence}'")
        print(f"Tokens: {tokens}\n")

    print("Sample BPE Vocabulary:", bpe_tokenizer.vocab[200:220])

    print("\n" + "="*60)
    print("WordPiece Tokenizer")
    print("="*60)
    wp_tokenizer = WordPieceTokenizer(vocab_size=K)
    wp_tokenizer.train(corpus)

    print("\nWordPiece Tokenization Examples:")
    for sentence in test_sentences:
        tokens = wp_tokenizer.tokenize(sentence)
        print(f"Sentence: '{sentence}'")
        print(f"Tokens: {tokens}\n")

    print("Sample WordPiece Vocabulary:", wp_tokenizer.vocab[200:220])


Byte-Pair Encoding (BPE) Tokenizer

BPE Tokenization Examples:
Sentence: 'Learning models need lots of data.'
Tokens: ['lea', 'r', 'ning</w>', 'mo', 'de', 'l', 's</w>', 'ne', 'ed</w>', 'lo', 'ts</w>', 'of</w>', 'da', 't', 'a</w>', '.']

Sentence: 'Tokenization helps language models handle new words.'
Tokens: ['to', 'ken', 'i', 'z', 'ation</w>', 'h', 'el', 'p', 's</w>', 'l', 'an', 'gu', 'ag', 'e</w>', 'mo', 'de', 'l', 's</w>', 'han', 'd', 'le</w>', 'new</w>', 'wor', 'ds</w>', '.']

Sample BPE Vocabulary: ['0m</w>', 'race</w>', 'ted</w>', 'pro', 'rop', 'won</w>', 'ru', 'lea', 'de', 'ken', 'med', 'ship', '7</w>', 'ke</w>', 'ere</w>', 'tion', 'se</w>', 'it', 'champion', 'championship']

WordPiece Tokenizer

WordPiece Tokenization Examples:
Sentence: 'Learning models need lots of data.'
Tokens: ['l', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'm', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'n', '[UNK]', '[UNK]', '[UNK]', 'l', '[UNK]', '[UNK]', '[UNK]', 'o', '[UNK]', 'd